## run model on sample of OpenSearch tweets

In [19]:
from bs4 import BeautifulSoup
import numpy as np
# import concurrent.futures
# from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoModel, AutoTokenizer
import torch, spacy,nltk,subprocess, json, requests,string,csv

model_name = "distilroberta-base"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# !python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
n_gram_range = (1, 2)
stop_words = "english"
embeddings=[]

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

import nltk, string, numpy as np
# nltk.download('punkt')

/opt/homebrew/Caskroom/mambaforge/base/envs/DT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
def get_npr_news(p):
    # Send a GET request to the NPR API
    r = requests.get("http://api.npr.org/query?apiKey=npr_key", params=p)

    # Parse the XML response to get the story URLs
    root = ET.fromstring(r.content)
    story_urls = [story.find('link').text for story in root.iter('story')]

    # For each story URL, send a GET request to get the HTML content
    full_stories = []
    for url in story_urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the main content of the story. This will depend on the structure of the webpage.
        # Here, we're assuming that the main content is in a <p> tag. You might need to adjust this depending on the webpage structure.
        story = soup.find_all('p')

        # Extract the text from the story
        full_story = ' '.join(p.text for p in story)
        full_stories.append(full_story)
    return full_stories


In [21]:
def chunk_text(text, max_len):
    # Tokenize the text into tokens
    tokens = nltk.word_tokenize(text)

    # Calculate the number of chunks and the size of the final chunk
    num_chunks = len(tokens) // max_len
    final_chunk_size = len(tokens) % max_len

    # If the final chunk is too small, distribute its tokens among the other chunks
    if final_chunk_size < max_len / 2:
        num_chunks += 1
        chunk_sizes = [len(tokens) // num_chunks + (1 if i < len(tokens) % num_chunks else 0) for i in range(num_chunks)]
        chunks = [tokens[sum(chunk_sizes[:i]):sum(chunk_sizes[:i+1])] for i in range(num_chunks)]
    else:
        chunks = [tokens[i:i + max_len] for i in range(0, len(tokens), max_len)]

    return chunks


In [22]:
# from torch.utils.data import DataLoader
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoModel, AutoTokenizer
import torch

def featurize_stories(text, max_len, top_k):
    # Extract candidate words/phrases
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([text])
    all_candidates = count.get_feature_names_out()
    doc = nlp(text)
    noun_phrases = set(chunk.text.strip().lower() for chunk in doc.noun_chunks)
    nouns = set()
    for token in doc:
        if token.pos_ == "NOUN":
            nouns.add(token.text)

    all_nouns = nouns.union(noun_phrases)
    candidates = list(filter(lambda candidate: candidate in all_nouns, all_candidates))
    candidate_tokens = tokenizer(candidates, padding=True, return_tensors="pt")
    # candidate_tokens = {k: v.to(device) for k, v in (candidate_tokens).items()}
    candidate_embeddings = model(**candidate_tokens)["pooler_output"]
    candidate_embeddings = candidate_embeddings.detach()#.to_numpy()

    # words = nltk.word_tokenize(text)
    # chunks = [words[i:i + 512] for i in range(0, len(words), 512)]
    chunks = chunk_text(text, max_len)  # use this to chunk better and use less padding thus less memory but also less affect from averging

    for chunk in chunks:
        text_tokens = tokenizer(chunk, padding=True, return_tensors="pt")
        text_tokens = {k: v.to(device) for k, v in (text_tokens).items()}
        text_embedding = model(**text_tokens)["pooler_output"]
        text_embedding = text_embedding.detach()#.to_numpy()
        embeddings.append(text_embedding)
    max_emb_shape = max(embedding.shape[0] for embedding in embeddings)
    padded_embeddings = [np.pad(embedding.cpu(), ((0, max_emb_shape - embedding.shape[0]), (0, 0))) for embedding in embeddings]
    avg_embedding = np.min(padded_embeddings, axis=0)
    distances = cosine_similarity(avg_embedding, candidate_embeddings.cpu())
    torch.cuda.empty_cache()
    return [candidates[index] for index in distances.argsort()[0][::-1][-top_k:]]



In [ ]:
import pandas as pd
# data=pd.read_csv('/content/drive/MyDrive/consult/Louie_disaster_tweets.csv',header=None)
data=pd.read_csv('florida-hurricane-tweet.csv')

In [ ]:
data.to_parquet('8.5k_florida_tweets.parquet')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet_features.txt',sep='\t')

In [ ]:
rank_articles=[]
from tqdm import tqdm
for i in tqdm(range(len(df),len(data['text']))):
    try:
        cc=featurize_stories(data['text'][i], max_len=512, top_k=4)
        # print(cc)
        rank_articles.append(cc)
    except IndexError:
        pass

In [ ]:
flattened_list = [item for sublist in rank_articles for item in sublist]
from collections import Counter
counter = Counter(flattened_list)
df = pd.DataFrame.from_dict(counter, orient='index', columns=['Count'])

df = df.sort_values(by='Count',ascending=False)
df.to_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet_features2.txt',sep='\t')
print(len(df))
# df[:25]

In [ ]:
filtered_list = [sublist for sublist in rank_articles if any('fire'.lower() in s.lower() for s in sublist)]
filtered_list

In [ ]:
import pandas as pd
df = pd.read_csv('florida-hurricane-tweet_features.txt',sep='\t')
df2=pd.read_csv('florida-hurricane-tweet_features2.txt',sep='\t')
df=pd.concat([df,df2])
print(df['Unnamed: 0'])
df = df.groupby('Unnamed: 0').sum().sort_values(by='Count',ascending=False)
df=df[df['Count']>int(np.round(len(df)*.001))]

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

# nouns = ['apple', 'John', 'London', 'dog', 'Mary', 'Paris', 'banana']
nouns= df.reset_index()['Unnamed: 0'].to_list()
doc = nlp(' '.join(nouns))

proper_nouns = [token.text for token in doc if token.pos_ == 'PROPN']

print(proper_nouns)

In [ ]:
print(len(proper_nouns))
proper_nouns

In [ ]:
len(nouns)

In [ ]:
i=['Iceland', '16-01-2024', ['Lava from a volcanic eruption in southwestern Iceland has streamed into a nearby town, engulfing homes and forcing the evacuation of local residents.\n', 'The Icelandic Meteorological Office said the eruption began just before 8:00 a.m. local time Sunday about a half mile from the town of Grindavík after a series of intense earthquakes.\n', 'A second fissure opened after noon Sunday and sent lava flows into the town, officials said.\n', 'Iceland\'s president, Guðni Th. Jóhannesson, said in an address to the nation on Sunday that a "daunting period of upheaval" had begun for those in the Reykjanes peninsula.\n', '"We continue to hope for as good an outcome as possible, in the face of these tremendous forces of nature," he said.\n', 'This is the second time in a month that a volcano has erupted just outside Grindavík, a coastal town about 25 miles from the Icelandic capital of Reykjavík.\n', 'An eruption on December 18 sent lava spewing into the air, but residents had already been told to leave due to heightened seismic activity.\n', "Iceland's government said Sunday that Grindavík had been evacuated early that morning and that the eruption isn't expected to reach other populated areas.\n", 'So far no flights have been disrupted, and officials are monitoring threats to infrastructure. At least three homes have either burned down or been overtaken by lava, according to the Icelandic broadcaster RUV.\n', "The government also said the eruption doesn't present a threat to life.\n", 'But in his speech, Jóhannesson offered his sympathy to the loved ones of Lúðvík Pétursson, a man who went missing in a work accident in Grindavík last week.\n', "According to Sky News, the 50-year-old Pétursson was filling crevasses formed by volcanic activity and earthquakes when he fell in a crack that had opened after last month's eruption.\n", 'Iceland is a hotspot for seismic activity, with 32 active volcanoes. A volcano erupts roughly every five years in the country, though eruptions have occurred more frequently recently. \n', ' Copyright 2024 NPR. To see more, visit https://www.npr.org.  ', '\n'], None, None, None]

## parameterize functions
 - parallelize text pulls from urls

In [ ]:
import argparse, signal,datetime,subprocess,json,os
from dotenv import load_dotenv
parser = argparse.ArgumentParser(description='Process OS data for dynamic features.')
parser.add_argument('-n', type=int, default=10, help='Number of data items to get')
parser.add_argument('-f', type=int, default=3, help='Number of features per item to get')
parser.add_argument('-o', type=str, default='dots_feats.csv', help='Output file name')
parser.add_argument('-p', type=int, default=1, help='Parallelize requests')
# parser.add_argument('-s', type=datetime, default=20230101, help='start date')
# parser.add_argument('-e', type=datetime, default=20231231, help='end date')
args, unknown = parser.parse_known_args()
load_dotenv()

In [ ]:
from dots_feat import get_OS_data
dd=get_OS_data(10)
dd

In [ ]:
os_url=([os.environ['OS_TOKEN']])
n=10
bash_command = f"""
curl -X GET "{os_url}/emergency-management-news/_search?scroll=1m" -H 'Content-Type: application/json' -d '{{
"_source": ["metadata.GDELT_DATE", "metadata.page_title","metadata.DocumentIdentifier", "metadata.Organizations","metadata.Persons","metadata.Themes","metadata.text", "metadata.Locations"],
    "size": {n},
    "query": {{
        "bool": {{
            "must": [
                {{"match_all": {{}}}}
            ]
        }}
    }}
}}'
"""
process = subprocess.run(bash_command, shell=True, capture_output=True, text=True)
output = process.stdout
data = json.loads(output)
data

In [ ]:
output

In [ ]:
# data = json.loads("input/feat_input.json")
with open("input/feat_input.json", 'r') as f:
    data = json.load(f)
data['_scroll_id']

In [ ]:
len(data['hits']['hits'])

In [ ]:
# Keep track of the scroll ID
scroll_id = response['_scroll_id']
es = OpenSearch([os.environ['OS_TOKEN']])
# Keep scrolling until no more results
while len(response['hits']['hits']):
    # print(response['hits']['hits'])  # process results

    response = es.scroll(
        scroll_id=scroll_id,
        scroll='1m'  # length of time to keep the scroll window open
    )

    # Update the scroll ID
    scroll_id = response['_scroll_id']


In [ ]:
from dots_feat import featurize_stories, process_data, get_OS_data, process_hit
data = get_OS_data(10)
data['hits']['hits']

In [ ]:
import sys,os,argparse,datetime
sys.argv = ['dots_feat.py', '-n', '10', '-f', '3', '-o', 'dots_feats.csv', '-p', '1']#, '-s', '20230101', '-e', '20231231']
parser = argparse.ArgumentParser(description='Process OS data for dynamic features.')
parser.add_argument('-n', type=int, default=10, help='Number of data items to get')
parser.add_argument('-f', type=int, default=3, help='Number of features per item to get')
parser.add_argument('-o', type=str, default='dots_feats.csv', help='Output file name')
parser.add_argument('-p', type=int, default=1, help='Parallelize requests')
# parser.add_argument('-s', type=datetime.date, default=datetime.datetime.strptime(20230101), help='start date')
# parser.add_argument('-e', type=datetime.date, default=datetime.datetime.strptime(20231231), help='end date')
args, unknown = parser.parse_known_args()

from dots.dots_feat import featurize_stories, process_data, get_OS_data, process_hit

In [ ]:
import dots

In [ ]:
s="20230101"
e="20231231"

s_date = datetime.datetime.strptime(s, "%Y%m%d")
e_date = datetime.datetime.strptime(e, "%Y%m%d")

# Convert datetime objects to ISO 8601 format
s_iso = s_date.isoformat() + "Z"
e_iso = e_date.isoformat() + "Z"

In [ ]:
[s_iso,e_iso]

In [ ]:
def get_OS_data(n=args.n):
    bash_command = f"""
    curl -X GET "{os_url}" -H 'Content-Type: application/json' -d '{{
"_source": ["metadata.GDELT_DATE", "metadata.page_title","metadata.DocumentIdentifier", "metadata.Organizations","metadata.Persons","metadata.Themes","metadata.text", "metadata.Locations"],
        "size": {n},
        "query": {{
            "bool": {{
                "must": [
                    {{"match_all": {{}}}},
                ]
            }}
        }}
    }}'
    """
    process = subprocess.run(bash_command, shell=True, capture_output=True, text=True)
    output = process.stdout
    data = json.loads(output)
    return data

In [ ]:
data = get_OS_data(10)
data

In [ ]:
data = get_OS_data(10)

In [ ]:
data = get_OS_data(10)
hits = data['hits']['hits']
hit = hits[1]
hit['_source']

from datetime import datetime
source = hit['_source']
date = datetime.strptime(source['metadata']['GDELT_DATE'], "%Y%m%d%H%M%S")
loc = source['metadata']['Locations']
# ex = None #source['metadata']['Extras']
title = source['metadata']['page_title']
url = source['metadata']['DocumentIdentifier']

In [ ]:
s = source['metadata']['Locations']

s = s.replace("'", '"')  # json requires double quotes for keys and string values
list_of_dicts = json.loads(s)


# location_full_name = source['metadata']['Locations']['Location FullName']

for dict in list_of_dicts:
    if 'Location FullName' in dict:
        print(dict['Location FullName'])

In [ ]:
data

In [ ]:

def process_data_fast(data):
    articles = []
    hits = data['hits']['hits']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_hit, hits))

    with open('output.csv', 'w') as file:
        writer = csv.writer(file)
        for text, soup,date,loc,title in results:
            articles.append([loc,loc,loc,date,title,title,text])
            # articles.append(text)

            writer.writerow([loc,loc,loc,date,title,title,text]) # force location into top feature, also assume title has important info
            # writer.writerow(text)
            writer.writerow(['\n'])

    with open('output.html', 'w') as file:
        file.write(str(results[-1][1]))  # Write the soup of the last hit

    # with open('output.csv', 'w') as file:
    #     file.write(str(results[-1][0]))  # Write the z of the last hit

    return articles


In [ ]:
from dots_feat import process_data_fast

In [ ]:
def process_hits(hit):
    source = hit['_source']
    GDELT_DATE = source['metadata']['GDELT_DATE']
    page_title = source['metadata']['page_title']
    url = source['metadata']['DocumentIdentifier']
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to get {url}")
        return z
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all(['p','div','a'])
    if not paragraphs:
        print(f"No <p> tags in {url}")
        return z

    for p in paragraphs:
        z.append(p.get_text())
    return z

with open('output.html', 'w') as file:
    file.write(str(soup))
    
with open('output.csv', 'w') as file:
    file.write(str(z))

# return articles
articles=[]
hits = data['hits']['hits']
for hit in hits:
    z=process_hits(hit)
    articles.append(z)
    

In [ ]:

def process_hit(hit):
    z = []
    source = hit['_source']
    GDELT_DATE = source['metadata']['GDELT_DATE']
    page_title = source['metadata']['page_title']
    location = source['metadata']["Locations"]
    extras = source['metadata']["Extras"]
    url = source['metadata']['DocumentIdentifier']

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for p in soup.find_all('p'):
        z.append(p.get_text())
    z.append(page_title)
    return z, soup

def process_data_fast(data):
    articles = []
    hits = data['hits']['hits']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_hit, hits))

    with open('output.csv', 'w') as file:
        writer = csv.writer(file)
        for z, soup in results:
            writer.writerow(z)
            writer.writerow(['\n'])
            articles.append(z)

    with open('output.html', 'w') as file:
        file.write(str(results[-1][1]))  # Write the soup of the last hit

    # with open('output.csv', 'w') as file:
    #     file.write(str(results[-1][0]))  # Write the z of the last hit

    return articles


In [ ]:

data['hits']['hits']

In [ ]:
data = get_OS_data(10)
# articles = process_data(data)
hits = data['hits']['hits']

In [ ]:
import concurrent.futures
results=[]
e = concurrent.futures.ThreadPoolExecutor()
    # try:
future = e.submit(process_hit, hits[1])
result = future.result(timeout=5)  # Set timeout for 5 seconds
results.append(result)

In [ ]:
data['hits']['hits'][1]['_source']

## process hits

In [ ]:
from datetime import datetime
import requests,os,csv,logging
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from opensearchpy import OpenSearch
load_dotenv()

In [ ]:
def process_response(response):
    hits = response["hits"]["hits"]
    output=[]
    for iii,hit in enumerate(hits):

        source = hit["_source"]
        text=[]
        try:
            date = datetime.strptime(source['metadata']['GDELT_DATE'], "%Y%m%d%H%M%S")
            date = formatted_date = date.strftime("%d-%m-%Y")
            loc = source['metadata']['Locations']
            loc = loc.replace("'", '"')  # json requires double quotes for keys and string values
            try:
                list_of_dicts = json.loads(loc)
                location_full_names = [dict['Location FullName'] for dict in list_of_dicts if 'Location FullName' in dict]
                loc = location_full_names[0]
            except:
                loc = None
            org = source['metadata']['Organizations']
            per = source['metadata']['Persons']
            theme = source['metadata']['Themes'].rsplit('_')[-1]
            title = source['metadata']['page_title']
            url = source['metadata']['DocumentIdentifier']
            output.append([date, loc, title, org, per, theme, url])
            # pagination_id=response['_scroll_id']
        
        except:
            # pagination_id=None
            pass
    return output
    

In [ ]:
response, client = get_massive_OS_data(args.n,args.t)

In [ ]:
# response['hits']['hits']
# response["_scroll_id"]

In [ ]:
articles=[]
hits = response["hits"]["hits"]
while len(hits) != 0:
    # try:
    response = client.scroll(
        scroll='5m',
        scroll_id=pagination_id
            )
    hits = response["hits"]["hits"]
    article = process_response(response)
    articles.append(article)
    # except:
    #     print("A ConnectionTimeout error occurred.")
    #     pass
    

In [ ]:
len(articles)

In [ ]:

flattened_list = [item for sublist in articles for item in sublist]

## scroll OS

In [ ]:
# os.getcwd()
# os.chdir('..')

In [ ]:
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from scrape import get_OS_data, get_massive_OS_data,get_google_news
from pull import process_hit, process_data,pull_data
os.getcwd()
# os.chdir("..")

In [ ]:
def get_scroll_data():
    response, client = get_massive_OS_data(1)
    pagination_id = response["_scroll_id"]
    hits = response["hits"]["hits"]
    articles=[]
    articles2=[]
    while len(hits) != 0 and len(articles2) < 11000:
        response = client.scroll(
            scroll='1m',
            scroll_id=pagination_id
                )
        hits = response["hits"]["hits"]
        # article = process_data(response)
        articles.append(hits)
        articles2 = [item for sublist in articles for item in sublist]
    return [item for sublist in articles for item in sublist]

In [ ]:
def extract_location(location_str):
    if location_str:
        try:
            location_list = json.loads(location_str.replace("'", '"'))
            return [dict['Location FullName'] for dict in location_list if 'Location FullName' in dict]
        except json.JSONDecodeError:
            return None
    else:
        return None

def process_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = []
    for p in paragraphs:
        text.append(p.get_text())
    return text

import json
import concurrent.futures, requests
from bs4 import BeautifulSoup

In [ ]:
data = [list(d['_source']['metadata'].values()) for d in articles]
df = pd.DataFrame(data, columns=['date','title', 'person', 'org', 'location', 'theme', 'text', 'url'])
df.date=pd.to_datetime(df.date).dt.strftime('%d-%m-%Y')
df['locc'] = df['location'].apply(extract_location)
with concurrent.futures.ThreadPoolExecutor() as executor:
    df['text'] = list(executor.map(process_url, df['url']))
return df.values.tolist()

## small gnews 

In [ ]:
import os
os.getcwd()
# os.chdir('..')

'/Users/apple/WRK/dcolinmorgan/dots/dots'

In [ ]:
from gnews import GNews
from tqdm import tqdm
from scrape import get_OS_data, get_massive_OS_data,get_google_news
from pull import process_hit, process_data,pull_data
# from feat import featurize_stories
google_news = GNews()

# google_news.period = '7d'  # News from last 7 days
google_news.max_results = 10000  # number of responses across a keyword
google_news.country = 'United States'  # News from a specific country 
google_news.language = 'english'  # News in a specific language
google_news.exclude_websites = ['yahoo.com', 'cnn.com']  # Exclude news from specific website i.e Yahoo.com and CNN.com
google_news.start_date = (2024, 1, 1) # Search from 1st Jan 2020
google_news.end_date = (2024, 3, 1) # Search until 1st March 2020



In [ ]:
json_resp = google_news.get_news('disaster')
article=[]

for i in tqdm(range(len(json_resp)), desc="grabbing directly from GoogleNews"):
    aa=(google_news.get_full_article(json_resp[i]['url']))
    try:
        date=aa.publish_date.strftime("%d-%m-%Y")
    except:
        date=None
    try:
        title=aa.title
        text=aa.text
    except:
        title=None
        text=None
    article.append([title,date,text])

# return article

In [ ]:
article

[['Individual Assistance',
  None,
  "FEMA can help support your recovery from a major disaster.\n\nGraphic FEMA Reforms Disaster Assistance FEMA implemented the most significant updates to disaster assistance in the last 20 years. These updates include: Flexible funding provided directly to survivors when they need it most.\n\nExpanded eligibility to help more people recover faster.\n\nSimplified application process to meet survivors’ individual needs. These changes apply to disasters declared on or after March 22, 2024. Learn More\n\nWhat Kind of Help Do You Need?\n\nI Need Immediate Assistance\n\nOur disaster assistance partners can provide help with immediate needs FEMA is not authorized to provide.\n\nEmergency Medical Assistance : Please dial 9-1-1.\n\n: Please dial 9-1-1. Emergency Shelter : Locate options by zip code by visiting the American Red Cross, or Salvation Army, or by texting SHELTER and your zip code (for example, “SHELTER 01234”) to 4FEMA (43362). For Spanish text RE

In [ ]:
# farticles = [item for sublist in article for item in sublist]
rank_articles=[]
for i in tqdm(article, desc="featurizing articles"):
    foreparts=str(i).split(',')[:2]  # location and date
    meat="".join(str(i).split(',')[2:-3])  # article text
    cc=featurize_stories(str(i), top_k = 3, max_len=512)
    rank_articles.append([foreparts,cc])

## lobstr API

In [1]:
import os,json
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
lobstr_key = os.environ['LOBSTR_KEY']

In [13]:
AA['id'][0]

'a9f051e3b2f44c6c99e8f81ff2eff552'

In [17]:
!curl 'https://api.lobstr.io/v1/runs/{AA['id'][0]}/stats' \
    -H 'Accept: application/json' \
    -H "Authorization: Token $lobstr_key" \

{"id": "a9f051e3b2f44c6c99e8f81ff2eff552", "object": "run", "duration": "0:08:03.927854", "ended_at": "2024-03-27T02:46:28", "eta": "\u221e", "is_done": true, "percent_done": "100%", "started_at": "2024-03-27T02:38:24", "total_requests": null, "total_results": 0, "total_tasks": 0, "total_tasks_done": 0, "total_tasks_left": 0, "current_task": "", "results_total": 0, "results_done": 0, "updated_at": "2024-03-27T02:38:24"}curl: (3) URL rejected: Malformed input to a URL function


In [3]:

!curl 'https://api.lobstr.io/v1/runs?page=1&page_size=3000' \
    -H 'Accept: application/json' \
    -H "Authorization: Token $lobstr_key" \
    -o 'input/runs.json'
    
with open("input/runs.json", 'r') as f:
    runs = json.load(f)
juns=pd.DataFrame(runs['data'])
AA=juns[['id','cluster','total_unique_results']]
latest_success_run = AA.loc[AA['total_unique_results'].ne(0).idxmax()]

!curl 'https://api.lobstr.io/v1/results?cluster=8de6e1bbf33f47b8bce451075b883252&run={latest_success_run['id']}&page=1&page_size=3000' \
    -H 'Accept: application/json' \
    -H "Authorization: Token $lobstr_key" \
    -o 'input/lobstr_results.json'
    
with open("input/lobstr_results.json", 'r') as f:
    data = json.load(f)
    
jata=pd.DataFrame(data['data'])
jata[['published_at','url','title','short_description']]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3369  100  3369    0     0   1742      0  0:00:01  0:00:01 --:--:--  1741
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1144k  100 1144k    0     0  23265      0  0:00:50  0:00:50 --:--:-- 1693744  0:00:19  0:00:25 26281:00:43  0:00:06 34810  0  0:00:50  0:00:48  0:00:02 19613


In [6]:
with open("input/lobstr_results.json", 'r') as f:
    data = json.load(f)
    
jata=pd.DataFrame(data['data'])
jata[['published_at','url','title','short_description']]

,published_at,url,title,short_description
0,2024-03-09T08:00:00Z,https://www.nytimes.com/2024/03/09/us/weather-...,Northeast Storm Expected to Bring Heavy Rain a...,"“Expect heavy rain, coastal flooding and stron..."
1,2024-02-29T08:00:00Z,https://www.cbsnews.com/sanfrancisco/news/how-...,How San Francisco schools are digging deep to ...,"On average, over 10 billion gallons of rain fa..."
2,2024-03-06T08:00:00Z,https://www.tehrantimes.com/news/495808/Global...,Global bodies show double standards in helping...,TEHRAN –International organizations have faile...
3,2024-03-21T07:00:00Z,https://disasterphilanthropy.org/disasters/202...,2024 US Tornadoes,2024 US Tornadoes ... The lack of a date-delin...
4,2024-03-23T14:53:00Z,https://en.antaranews.com/news/309147/polri-pr...,Polri provides trauma healing services to floo...,The National Police (Polri) provided trauma he...
...,...,...,...,...
1777,2024-03-05T08:00:00Z,https://www.foxweather.com/watch/play-6fcee004...,Coastal storms soak East Coast as next storm r...,Back-to-back coastal storms are set to soak th...
1778,2024-03-01T08:00:00Z,https://www.essentiallysports.com/nascar-news-...,Las Vegas Weather Update: Severe Thunderstorm ...,"Well, it's pretty obvious. The Las Vegas Motor..."
1779,2024-02-29T08:00:00Z,https://www.theguardian.com/world/2024/feb/29/...,Texas wildfires cause chaos as largest blaze i...,Governor issues disaster declaration as emerge...
1780,2024-03-05T08:00:00Z,https://www.millenniumpost.in/bengal/state-lau...,State launches first-of-its-kind upgraded temp...,Kolkata: State Disaster Management and Civil D...


## lobstr via gdocs

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/178sqEWzqubH0znhx7Z6u9ig2EjCRvl0dUsA7b6hQpmY/export?format=csv'
import pandas as pd
df = pd.read_csv(url)
df=df[['published_at','url','title','short_description']]

In [ ]:
articles = pd.read_csv('input/lobstr_text.txt', sep='\t', sep='\t', index_col=0, names=['text'], skiprows=1)
url = 'https://docs.google.com/spreadsheets/d/178sqEWzqubH0znhx7Z6u9ig2EjCRvl0dUsA7b6hQpmY/export?format=csv'
df = pd.read_csv(url)
len(articles) == len(df)
# len(articles)

True

In [ ]:
articles.to_parquet('input/lobstr_text.parquet', index=False)#, mode='a')

In [ ]:
jj=pd.read_parquet('input/lobstr_text.parquet')

In [ ]:
jj

,0
0,None
1,[]
2,"['MARCH 4, 2024 — A 7.8 magnitude earthquake d..."
3,['\n Please contact us ...
4,[]
...,...
4139,['The group of skiers were missing for more th...
4140,"['Published By: Shankhyaneel Sarkar', 'AFP', ..."
4141,[]
4142,['A family of seven who went missing in region...


In [ ]:
import torch
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == 'cuda':
    dataloader = DataLoader(data['text'], batch_size=1, shuffle=True, num_workers=4)
    RR = dataloader
else:
    RR = articles
for j,i in tqdm(enumerate(RR), total=len(RR), desc="featurizing articles"):

# for i in tqdm(articles, desc="featurizing articles"):
    try:
        foreparts = str(i).split(',')[:2]  # location and date
    except:
        foreparts=None
    meat="".join(str(j).split(',')[2:-3])  # text
    try:
        cc=featurize_stories(str(i), top_k = args.f, max_len=512)
    except:
        'damn'

featurizing articles: 100%|██████████| 3078/3078 [00:00<00:00, 730248.75it/s]


In [ ]:
RR

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [ ]:
# from multiprocessing import Pool
from tqdm import tqdm
import requests, concurrent
from bs4 import BeautifulSoup
# def handler(signum, frame):
    # raise TimeoutError()/
# signal.signal(signal.SIGALRM, handler)


def process_url(url):
    try:
        # signal.alarm(5)
        response = requests.get(url,timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all(['p'])
        text = []
        for p in paragraphs:
            text.append(p.get_text())
        # signal.alarm(0)
        return text
    except Exception as e:
        # print(f"Error processing URL {url}: {e}")
        return None
    

In [ ]:
# results=[]
# for i in tqdm(df['url'], desc="grabbing text from url"):
#     result = process_url(i)
#     results.append(result)
# df['text'] = results

In [ ]:
results=[]
with concurrent.futures.ThreadPoolExecutor() as executor:
    result = list(tqdm(executor.map(process_url, df['url']), total=len(df['url'])))

In [ ]:
# farticles = [item for sublist in article for item in sublist]
rank_articles=[]
for i in tqdm(result, desc="featurizing articles"):
    foreparts = df[['published_at','title']]
    try:
        cc=featurize_stories(str(i), top_k = 3, max_len=512)
    except:
        cc=None
        pass
    rank_articles.append([foreparts,cc])

featurizing articles: 100%|██████████| 2682/2682 [1:28:12<00:00,  1.97s/it]


In [ ]:
pd.DataFrame(rank_articles)

,0,1
0,published_at \ 0 2024-02-...,None
1,published_at \ 0 2024-02-...,"[huge stresses, disasters, aspects]"
2,published_at \ 0 2024-02-...,None
3,published_at \ 0 2024-02-...,"[surveillance, candidates, surveillance towers]"
4,published_at \ 0 2024-02-...,"[judgement, mercedes, catastrophe]"
...,...,...
2677,published_at \ 0 2024-02-...,"[temperatures, widespread rains, meteorologist]"
2678,published_at \ 0 2024-02-...,"[afternoon, thunderstorms, severe thunderstorms]"
2679,published_at \ 0 2024-02-...,"[snowfall, morning, winter weather]"
2680,published_at \ 0 2024-02-...,"[meteorologist, heavy rain, gusty winds]"


In [ ]:
# farticles = [item for sublist in article for item in sublist]
rank_articles=[]
for i in tqdm(df['text'], desc="featurizing articles"):
    foreparts = df['datepublished_at','title'].tolist()
    cc=featurize_stories(str(i), top_k = 3, max_len=512)
    rank_articles.append([foreparts,cc])